In [1]:
import torch
from model import MultiSeqBase
model = MultiSeqBase(n_labels=3, hidden_size=32, embedding_dim=4, dropout_p=0.2)
optim = torch.optim.Adam(model.parameters(), weight_decay=1e-3)

# Load model and optimizer
checkpoint = torch.load('test_model_and_optimizer.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])

In [4]:
from utils import db2Rseq, db2Hseq, dt2T, db2S, y2db

r_seq = db2Rseq()
h_seq = db2Hseq()
t = dt2T()
s_attrs, s_embed = db2S()

output = model(r_seq.float(), h_seq.float(), t.int(), s_attrs.float())

In [6]:
output, output.shape

(tensor([[-0.1165, -2.2149, -7.1474],
         [-0.0848, -2.5178, -7.3008],
         [-0.1128, -2.2417, -7.9024],
         ...,
         [-0.1432, -2.0179, -7.6878],
         [-0.0704, -2.7311, -5.8505],
         [-0.0855, -2.5200, -6.5292]], grad_fn=<LogSoftmaxBackward0>),
 torch.Size([722, 3]))

In [3]:
# get station id from database
from utils import db_connect, db2Sid

sid = db2Sid()

In [16]:
def y2db(outputs):
    sid = db2Sid()
    update_query = "UPDATE occupancy SET Occupancy_20 = %s WHERE Sid = %s"
    for values in list(zip(np.array(output), sid)):
        
        with connection.cursor() as cursor:
            cursor.execute(update_query, values)
        
    connection.commit()
    connection.close()

In [18]:
connection = db_connect()
connection.close()

In [16]:
pd.read_csv('data/station_embed.csv')

,id,sid,umap_1,umap_2,umap_3,umap_4,umap_5,umap_6,umap_7,umap_8
0,0,강원_강릉_강남동주민센터,8.261596,5.375485,4.711554,6.793701,6.844904,9.262389,7.546947,6.304873
1,2,강원_강릉_강릉축협 하나로마트,8.151678,5.585845,4.004126,5.723880,5.575205,9.365921,6.776956,6.366620
2,3,강원_강릉_강원도교육연수원,8.329152,5.443759,4.051734,2.785187,4.890255,7.995803,6.954027,5.969278
3,4,강원_강릉_강원도립대학,8.160399,5.584676,3.984990,2.761592,4.956375,8.050358,6.829950,5.891249
4,5,강원_강릉_강원양돈농협 하나로마트,8.280510,5.424000,4.090035,5.529007,5.678575,9.319400,6.978778,6.485642
...,...,...,...,...,...,...,...,...,...,...
717,1057,충북_충주_이마트 충주점,8.107466,5.625983,3.874539,5.608638,5.279572,8.857050,6.555145,6.166920
718,1058,충북_충주_충주시청 민원주차장,8.612368,4.834136,4.443982,4.470196,5.598176,8.512796,7.306812,6.360420
719,1059,충북_충주_충주실내체육관,7.992418,5.341586,4.796701,6.337998,6.943312,8.694623,7.003403,6.382822
720,1060,충북_충주_충주종합스포츠타운,6.554224,5.846991,3.563491,-0.446774,4.698729,8.630402,6.800158,5.953447
